In [3]:
import pandas as pd
import numpy as np
import sys
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split

In [3]:
data.train

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [4]:
data.test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
df = pd.read_csv('./Netflix_rating_reduced.csv')

In [6]:
trainset, testset = train_test_split(df, stratify=df['User_ID'], test_size=0.3, random_state=88)

In [8]:
trainset.head()

,User_ID,Rating,Movie_ID
2572280,77691,4,3605
1062272,78610,2,1561
1046686,323908,3,1542
2021556,351559,4,2862
2462634,25375,3,3427


In [9]:
testset.head()

,User_ID,Rating,Movie_ID
2422973,328775,3,3371
3101138,377464,3,4284
527664,430616,3,723
646186,59882,5,907
1915771,198638,4,2690


In [11]:
train = trainset.pivot_table(trainset, index = 'User_ID', columns = 'Movie_ID')
test = testset.pivot_table(testset, index = 'User_ID', columns = 'Movie_ID')

In [13]:
test.head()

Rating                                               ...            \
Movie_ID   3    8    16   17   18   26   28   30   32   33    ... 4472 4474   
User_ID                                                       ...             
6           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
7           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
79          NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  NaN  NaN  ...  NaN  NaN   
97          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
134         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

                                                  
Movie_ID 4478 4479 4485 4488 4490 4492 4493 4496  
User_ID                                           
6         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
7         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
79        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
97        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
134       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 1350 columns]

In [14]:
train.fillna(0.0, inplace = True)
test.fillna(0.0, inplace = True)

In [15]:
test.head()

Rating                                               ...            \
Movie_ID   3    8    16   17   18   26   28   30   32   33    ... 4472 4474   
User_ID                                                       ...             
6           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
7           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
79          0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  ...  0.0  0.0   
97          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
134         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                  
Movie_ID 4478 4479 4485 4488 4490 4492 4493 4496  
User_ID                                           
6         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
79        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
97        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
134       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 1350 columns]

In [16]:
train = train.to_numpy()
test = test.to_numpy()

In [17]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
class MatrixFactorizaion():
    def __init__(self, train, test, k, learning_rate, reg_param, epochs, verbose = False):
        self.R = train
        self.test = test
        self.num_users, self.num_items = train.shape
        self.k = k
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.epochs = epochs
        self.verbose = verbose
        
    def fit(self):
        #initialize latent mat
        self._P = np.random.normal(scale = 1./self.k, size = (self.num_users, self.k))
        self._Q = np.random.normal(scale = 1./self.k, size = (self.num_items, self.k))
        
        #initialize biases
        self._b_P = np.zeros(self.num_users)
        self._b_Q = np.zeros(self.num_items)
        self._b = np.mean(self.R[np.where(self.R != 0)])
        
        self.training_epochs = []
        start = timer()
        
        for epoch in range(self.epochs):
            
            for u in range(self.num_users):
                for i in range(self.num_items):
                    if self.R[u,i] > 0:
                        self.gradient_descent(u,i, self.R[u,i])
                        
                
            train_cost, test_cost = self.cost()
            self.training_epochs.append((epoch, train_cost, test_cost))
            
            if self.verbose == True and ((epoch + 1) % 10 == 0):
                print('iter: %d, train_cost = %.4f, test_cost = %.4f' % (epoch+1, train_cost, test_cost))
                
        print('time: %.4f seconds' % (timer()-start))
    
    #RMSE computation
    def cost(self):
        xi, yi = self.R.nonzero()
        test_x, test_y = self.test.nonzero()
        predicted = self.get_complete_matrix()
        cost_train = 0; cost_test = 0
        
        for x, y in zip(xi,yi):
            cost_train += pow(self.R[x,y] - predicted[x,y], 2)
        for i,j in zip(test_x, test_y):
            cost_test += pow(self.test[i,j] - predicted[i,j], 2)
            
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
    def gradient(self, error, u, i):
        dp = (error * self._Q[i,:]) - (self.reg_param * self._P[u,:])
        dq = (error * self._P[u,:]) - (self.reg_param * self._Q[i,:])
        return dp, dq
    
    def gradient_descent(self, u, i, rating):
        prediction = self.get_prediction(u,i)
        error = rating - prediction
        
        self._b_P[u] += self.learning_rate * (error - self.reg_param * self._b_P[u])
        self._b_Q[i] += self.learning_rate * (error - self.reg_param * self._b_Q[i])
        
        dp, dq = self.gradient(error, u, i)
        self._P[u,:] += self.learning_rate * dp
        self._Q[i,:] += self.learning_rate * dq
        
    def get_prediction(self, u, i):
        return self._b + self._b_P[u] + self._b_Q[i] + self._P[u,:].dot(self._Q[i,:].T)
    
    def get_complete_matrix(self):
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis, :] + self._P.dot(self._Q.T)
    
    def print_result(self):
        print('final R predicted: \n')
        print(self.get_complete_matrix())
        print('final RMSE:')
        print(self.training_epochs[self.epochs-1][2])

In [19]:
np.random.seed(47)

factorize = MatrixFactorizaion(train, test, k=40, learning_rate=0.01, reg_param=0.01, epochs=10, verbose=True)
factorize.fit()
factorize.print_result()

iter: 10, train_cost = 0.7287, test_cost = 0.8579
time: 665.0961 seconds
final R predicted: 

[[3.13329704 2.37954246 3.01771297 ... 2.04870891 2.65679332 2.9617655 ]
 [4.32999995 3.58519586 3.76734081 ... 2.84556602 3.63646451 4.41370155]
 [3.48897213 2.8900205  2.97058837 ... 1.53585096 2.7901268  3.15359593]
 ...
 [2.59672119 2.52024498 2.52107382 ... 1.59479263 2.05788412 2.17945777]
 [4.1658815  3.52069446 3.48243663 ... 3.20594595 3.64355498 3.92071594]
 [3.70125052 3.05717655 3.44920446 ... 2.63466918 3.00020108 3.4267222 ]]
final RMSE:
0.8578886104126082


In [23]:
test[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])